# NEWS SENTIMENT ANALYSIS

In [ ]:
!pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.33.0
    Uninstalling accelerate-0.33.0:
      Successfully uninstalled accelerate-0.33.0


In [ ]:
!pip install --upgrade pip

  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
!pip install --upgrade datasets transformers accelerate evaluate jiwer tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 165.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 100.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.0
    Uninstalling tensorboard-2.17.0:
      Successfully uninstalled tensorboard-2.17.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0

In [ ]:
!pip install evaluate rouge_score transformers[torch]

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=dd72abd746674a1ae6a6c58dc2fc75d4a45a562952c11b0f71c6e0db0978c33f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
# Importing libraries for data preparation and visualization
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Importing warnings to ignore them
import warnings
warnings.filterwarnings("ignore")

# Importing libraries for pickle, json, and joblib files
import pickle
import json
import joblib
import copy

# Importing train_test_split for data splitting
from sklearn.model_selection import train_test_split

In [ ]:
from datasets import load_dataset, DatasetDict, concatenate_datasets, Dataset

# HOSHI DATASET FOR SENTIMENT

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
parent_folder = "/content/drive/MyDrive/final project/"
dataset = pd.read_csv(parent_folder+'hoshi_sentiment_dataset.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset.head()

,news_title,reddit_title,sentiment,text,url
0,Mark Cuban launches generic drug company,Billionaire Mark Cuban just launched a drug co...,1.0,Billionaire investor and Shark Tank star Mark ...,https://www.beckershospitalreview.com/pharmacy...
1,From Defendant to Defender: One Wrongfully Con...,"Man falsely imprisoned for 10 years, uses pris...",1.0,Attorney Jarrett Adams recently helped overtur...,https://www.nbcnews.com/news/us-news/defendant...
2,"Amazon Tribe Wins Lawsuit Against Big Oil, Sav...",Amazon tribe wins legal battle against oil com...,1.0,The Amazon Rainforest is well known across the...,https://www.disclose.tv/amazon-tribe-wins-laws...
3,Newark police: No officer fired a single shot ...,Newark police: No officer fired a single shot ...,1.0,Newark police: No officer fired a single shot ...,https://newjersey.news12.com/newark-police-no-...
4,Ingen barn døde i trafikken i 2019,No children died in traffic accidents in Norwa...,1.0,I 1970 døde det 560 mennesker i den norske tra...,https://www.nrk.no/trondelag/ingen-barn-dode-i...


In [ ]:
dataset['sentiment'].value_counts()

,count
sentiment,
1,748
0,100


In [ ]:
dataset.sentiment = dataset.sentiment.astype(int)

In [ ]:
df_train, df_test = train_test_split(dataset, test_size=0.3, stratify=dataset['sentiment'], random_state=123)

In [ ]:
df_test, df_valid = train_test_split(df_test, test_size=0.5, stratify=df_test['sentiment'], random_state=123)

In [ ]:
# Convert the DataFrames to Datasets
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)
valid_dataset = Dataset.from_pandas(df_valid)

# Create the DatasetDict
dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'valid': valid_dataset
})

# Verify the structure
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['news_title', 'reddit_title', 'sentiment', 'text', 'url', '__index_level_0__'],
        num_rows: 593
    })
    test: Dataset({
        features: ['news_title', 'reddit_title', 'sentiment', 'text', 'url', '__index_level_0__'],
        num_rows: 127
    })
    valid: Dataset({
        features: ['news_title', 'reddit_title', 'sentiment', 'text', 'url', '__index_level_0__'],
        num_rows: 128
    })
})


In [ ]:
dataset_run = dataset_dict.remove_columns(['text', 'reddit_title', 'url', '__index_level_0__'])

In [ ]:
def rename_column(example):
    example['text'] = example.pop('news_title')
    example['label'] = example.pop('sentiment')  # Rename the column
    return example

# Apply the renaming to each split in the dataset
for split in dataset_run.keys():
    dataset_run[split] = dataset_run[split].map(rename_column)

Map:   0%|          | 0/593 [00:00<?, ? examples/s]

Map:   0%|          | 0/127 [00:00<?, ? examples/s]

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

In [ ]:
dataset_run

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 593
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 127
    })
    valid: Dataset({
        features: ['text', 'label'],
        num_rows: 128
    })
})

In [ ]:
dataset_run['train'][0]

{'text': 'Teacher with criminal history arrested after altercation with student',
 'label': 0}

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)

tokenized_train = dataset_run['train'].map(preprocess_function, batched=True)
tokenized_test = dataset_run['test'].map(preprocess_function, batched=True)
tokenized_valid = dataset_run['valid'].map(preprocess_function, batched=True)

Map:   0%|          | 0/593 [00:00<?, ? examples/s]

Map:   0%|          | 0/127 [00:00<?, ? examples/s]

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
from datasets import load_metric
from torch.nn import BCEWithLogitsLoss # import the function

def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")

   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import TrainingArguments, Trainer

repo_name = "./distil_news_finetune2"

training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   metric_for_best_model="accuracy",
   push_to_hub=True
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_valid,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=76, training_loss=0.11719746338693719, metrics={'train_runtime': 10.7467, 'train_samples_per_second': 110.36, 'train_steps_per_second': 7.072, 'total_flos': 9577496412984.0, 'train_loss': 0.11719746338693719, 'epoch': 2.0})

In [ ]:
trainer.evaluate(tokenized_test)

{'eval_loss': 0.20498381555080414,
 'eval_accuracy': 0.9448818897637795,
 'eval_f1': 0.9696969696969697,
 'eval_runtime': 2.0649,
 'eval_samples_per_second': 61.505,
 'eval_steps_per_second': 3.874,
 'epoch': 2.0}

In [ ]:
tokenizer.push_to_hub("distil_news_finetune2")

CommitInfo(commit_url='https://huggingface.co/anggari/distil_news_finetune2/commit/583f6beb728066a88efcde2572a7b8d71beef65b', commit_message='Upload tokenizer', commit_description='', oid='583f6beb728066a88efcde2572a7b8d71beef65b', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import pipeline

sentiment_model = pipeline(model="anggari/distil_news_finetune2")

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
sentiment_model(["""
A busy bridge in northern Vietnam has collapsed in the wake of super typhoon Yagi, plunging 10 cars and two scooters into the Red River, Deputy Prime Minister Ho Duc Phoc said on Monday.
At least three people have so far been rescued and 13 are missing after the Phong Chau bridge in Phu Tho province collapsed, Mr Ho added. It is unclear if there have been any deaths.
Part of the 375-metre (1230 feet) structure is still standing, and the military has been instructed to build a pontoon bridge as soon as possible.
Yagi, Asia's most powerful storm this year, has killed more than 60 people since it made landfall in Vietnam on Saturday, bringing strong winds of up to 203 km/h (126 mph).
In the days since, the storm has wreaked havoc in northern Vietnam.At least 44 victims were killed by landslides and flash floods, the country's Ministry of Agriculture and Rural Development said on Monday - a 68-year-old woman, a one-year-old boy and a newborn baby among them.
More than 240 people have been injured, while some 1.5 million are still without power.
The typhoon also tore roofs from buildings and uprooted trees. Although it has weakened into a tropical depression, authorities have warned of more flooding and landslides as the storm moves westwards.
In the Yen Bai province, flood waters reached a metre (three feet) high on Monday, with 2,400 families moved to higher ground as the water levels rose, AFP news agency reported.
Yagi also sunk and swept adrift dozens of fishing boats. On Sunday, search and rescue personnel found 27 people drifting at sea after a dozen fishermen were reported missing.
Nearly 50,000 people have been evacuated from coastal towns in Vietnam, with authorities issuing a warning to remain indoors.
Schools were temporarily closed in 12 northern provinces, including Hanoi.
Before hitting Vietnam, Yagi left 24 people dead across southern China and the Philippines.
Scientists say typhoons and hurricanes are becoming stronger, more frequent and staying over land for longer due to climate change. Warmer ocean waters mean storms pick up more energy, which leads to higher wind speeds.
A warmer atmosphere also holds more moisture, which can lead to more intense rainfall.
"""
])

[{'label': 'LABEL_0', 'score': 0.664634644985199}]

In [ ]:
sentiment_model(["""
I love you
"""
])

[{'label': 'LABEL_1', 'score': 0.5964159965515137}]

# SEHYUN DATASET FOR SENTIMENT

In [ ]:
ds = load_dataset("sehyun66/News-sentiments", "distill")

Generating default split:   0%|          | 0/316086 [00:00<?, ? examples/s]

In [ ]:
ds

DatasetDict({
    default: Dataset({
        features: ['headline', 'summary', 'headline_sentiment', 'summary_sentiment'],
        num_rows: 316086
    })
})

In [ ]:
ds['default'][0]

{'headline': 'Notable companies reporting after market close',
 'summary': 'Looking for stock market analysis and research with proves results? Zacks.com offers in-depth financial research with over 30years of proven results.',
 'headline_sentiment': {'postive': '0.25946614146232605',
  'negative': '0.3799891471862793',
  'neutral': '0.25364914536476135'},
 'summary_sentiment': {'postive': '0.2572893500328064',
  'negative': '0.37214919924736023',
  'neutral': '0.2266107201576233'}}

In [ ]:
# Function to determine the dominant sentiment
def categorize_sentiment(sentiment_scores):
    positive = sentiment_scores['postive']
    negative = sentiment_scores['negative']
    neutral = sentiment_scores['neutral']

    if positive >= negative and positive >= neutral:
        return 'positive', 1  # Positive
    elif negative >= positive and negative >= neutral:
        return 'negative', 2  # Negative
    else:
        return 'neutral', 0  # Neutral

# Function to apply sentiment categorization
def apply_sentiment_categorization(example):
    sentiment_word, sentiment_value = categorize_sentiment(example['summary_sentiment'])
    example['summary_sentiment_word'] = sentiment_word
    example['summary_sentiment_value'] = sentiment_value
    return example

# Apply the categorization to all rows in the dataset
ds['default'] = ds['default'].map(apply_sentiment_categorization)

# Output the updated dataset
print(ds)

Map:   0%|          | 0/316086 [00:00<?, ? examples/s]

DatasetDict({
    default: Dataset({
        features: ['headline', 'summary', 'headline_sentiment', 'summary_sentiment', 'summary_sentiment_word', 'summary_sentiment_value'],
        num_rows: 316086
    })
})


In [ ]:
# Function to apply sentiment categorization
def apply_hdsentiment_categorization(example):
    sentiment_word, sentiment_value = categorize_sentiment(example['summary_sentiment'])
    example['summary_hdsentiment_word'] = sentiment_word
    example['summary_hdsentiment_value'] = sentiment_value
    return example

# Apply the categorization to all rows in the dataset
ds['default'] = ds['default'].map(apply_hdsentiment_categorization)

# Output the updated dataset
print(ds)

Map:   0%|          | 0/316086 [00:00<?, ? examples/s]

DatasetDict({
    default: Dataset({
        features: ['headline', 'summary', 'headline_sentiment', 'summary_sentiment', 'summary_sentiment_word', 'summary_sentiment_value', 'summary_hdsentiment_word', 'summary_hdsentiment_value'],
        num_rows: 316086
    })
})


In [ ]:
sentiment_mapping = {
    1: 'positive',
    0: 'neutral',
    2: 'negative',
}

def convert_sentiment_to_label(example):
    example['summary_sentiment_label'] = sentiment_mapping.get(example['summary_sentiment_value'], 'unknown')
    return example

# Apply the mapping to the dataset
ds['default'] = ds['default'].map(convert_sentiment_to_label)

Map:   0%|          | 0/316086 [00:00<?, ? examples/s]

In [ ]:
ds['default'][0]

{'headline': 'Notable companies reporting after market close',
 'summary': 'Looking for stock market analysis and research with proves results? Zacks.com offers in-depth financial research with over 30years of proven results.',
 'headline_sentiment': {'postive': '0.25946614146232605',
  'negative': '0.3799891471862793',
  'neutral': '0.25364914536476135'},
 'summary_sentiment': {'postive': '0.2572893500328064',
  'negative': '0.37214919924736023',
  'neutral': '0.2266107201576233'},
 'summary_sentiment_word': 'negative',
 'summary_sentiment_value': 2,
 'summary_hdsentiment_word': 'negative',
 'summary_hdsentiment_value': 2,
 'summary_sentiment_label': 'negative'}

In [ ]:
ds_pd=pd.DataFrame(ds['default'])

In [ ]:
ds_pd['summary_sentiment_word'].value_counts()

,count
summary_sentiment_word,
negative,303721
positive,12362
neutral,3


In [ ]:
ds = ds['default'].train_test_split(test_size=0.4, seed = 21)#, stratify_by_column="summary_sentiment_label")
ds_test = ds['test'].train_test_split(test_size=0.5, seed=21)#, stratify_by_column="summary_sentiment_label")
dataset = DatasetDict({
    'train': ds['train'],
    'test': ds_test['test'],
    'valid':ds_test['train']})
# see structure
dataset

DatasetDict({
    train: Dataset({
        features: ['headline', 'summary', 'headline_sentiment', 'summary_sentiment', 'summary_sentiment_word', 'summary_sentiment_value', 'summary_hdsentiment_word', 'summary_hdsentiment_value', 'summary_sentiment_label'],
        num_rows: 189651
    })
    test: Dataset({
        features: ['headline', 'summary', 'headline_sentiment', 'summary_sentiment', 'summary_sentiment_word', 'summary_sentiment_value', 'summary_hdsentiment_word', 'summary_hdsentiment_value', 'summary_sentiment_label'],
        num_rows: 63218
    })
    valid: Dataset({
        features: ['headline', 'summary', 'headline_sentiment', 'summary_sentiment', 'summary_sentiment_word', 'summary_sentiment_value', 'summary_hdsentiment_word', 'summary_hdsentiment_value', 'summary_sentiment_label'],
        num_rows: 63217
    })
})

In [ ]:
df_train=pd.DataFrame(dataset['train'])
df_test=pd.DataFrame(dataset['test'])
df_valid=pd.DataFrame(dataset['valid'])

In [ ]:
df_train.head()

,headline,summary,headline_sentiment,summary_sentiment,summary_sentiment_word,summary_sentiment_value,summary_hdsentiment_word,summary_hdsentiment_value
0,WSFS Financial Becomes Oversold,Looking for stock market analysis and research...,"{'postive': '0.2364639937877655', 'negative': ...","{'postive': '0.2572893500328064', 'negative': ...",negative,2,negative,2
1,Qualcomm price target lowered to $150 from $16...,Looking for stock market analysis and research...,"{'postive': '0.2528002858161926', 'negative': ...","{'postive': '0.2572893500328064', 'negative': ...",negative,2,negative,2
2,Workday Gets Price Target Hikes By Analysts Fo...,Looking for stock market analysis and research...,"{'postive': '0.23863041400909424', 'negative':...","{'postive': '0.2572893500328064', 'negative': ...",negative,2,negative,2
3,UBS Sticks to Their Hold Rating for Sandfire R...,Looking for stock market analysis and research...,"{'postive': '0.2587398588657379', 'negative': ...","{'postive': '0.2572893500328064', 'negative': ...",negative,2,negative,2
4,Vor Biopharma (VOR) Gets a Buy from JMP Securi...,Looking for stock market analysis and research...,"{'postive': '0.24814671277999878', 'negative':...","{'postive': '0.2572893500328064', 'negative': ...",negative,2,negative,2


In [ ]:
df_train['summary_sentiment_value'].unique()

array([2, 1, 0])

In [ ]:
df_train['summary_sentiment_word'].value_counts()

,count
summary_sentiment_word,
negative,182345
positive,7305
neutral,1


# DISTILBERT

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['headline', 'summary', 'headline_sentiment', 'summary_sentiment', 'summary_sentiment_word', 'summary_sentiment_value', 'summary_hdsentiment_word', 'summary_hdsentiment_value', 'summary_sentiment_label'],
        num_rows: 189651
    })
    test: Dataset({
        features: ['headline', 'summary', 'headline_sentiment', 'summary_sentiment', 'summary_sentiment_word', 'summary_sentiment_value', 'summary_hdsentiment_word', 'summary_hdsentiment_value', 'summary_sentiment_label'],
        num_rows: 63218
    })
    valid: Dataset({
        features: ['headline', 'summary', 'headline_sentiment', 'summary_sentiment', 'summary_sentiment_word', 'summary_sentiment_value', 'summary_hdsentiment_word', 'summary_hdsentiment_value', 'summary_sentiment_label'],
        num_rows: 63217
    })
})

In [ ]:
dataset_run = dataset.remove_columns(["headline", 'headline_sentiment',
                                      'summary_hdsentiment_word','summary_sentiment_label',
                                      'summary_hdsentiment_value','summary_sentiment',
                                      'summary_sentiment_word'])

In [ ]:
dataset_run

DatasetDict({
    train: Dataset({
        features: ['summary', 'summary_sentiment_value'],
        num_rows: 189651
    })
    test: Dataset({
        features: ['summary', 'summary_sentiment_value'],
        num_rows: 63218
    })
    valid: Dataset({
        features: ['summary', 'summary_sentiment_value'],
        num_rows: 63217
    })
})

In [ ]:
def rename_column(example):
    example['text'] = example.pop('summary')
    example['label'] = example.pop('summary_sentiment_value')  # Rename the column
    return example

# Apply the renaming to each split in the dataset
for split in dataset_run.keys():
    dataset_run[split] = dataset_run[split].map(rename_column)

Map:   0%|          | 0/189651 [00:00<?, ? examples/s]

Map:   0%|          | 0/63218 [00:00<?, ? examples/s]

Map:   0%|          | 0/63217 [00:00<?, ? examples/s]

In [ ]:
dataset_run

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 189651
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 63218
    })
    valid: Dataset({
        features: ['text', 'label'],
        num_rows: 63217
    })
})

In [ ]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)

tokenized_train = dataset_run['train'].map(preprocess_function, batched=True)
tokenized_test = dataset_run['test'].map(preprocess_function, batched=True)
tokenized_valid = dataset_run['valid'].map(preprocess_function, batched=True)

Map:   0%|          | 0/189651 [00:00<?, ? examples/s]

Map:   0%|          | 0/63218 [00:00<?, ? examples/s]

Map:   0%|          | 0/63217 [00:00<?, ? examples/s]

In [ ]:
tokenized_train

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 189651
})

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")

   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels, average='macro')["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import TrainingArguments, Trainer

repo_name = "./distil_news_finetune"

training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   metric_for_best_model="accuracy",
   push_to_hub=True
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_valid,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Step,Training Loss
500,0.026400
1000,0.007200
1500,0.006000
2000,0.003600
2500,0.005800
3000,0.002000
3500,0.002700
4000,0.007500
4500,0.003000
5000,0.001800


TrainOutput(global_step=23708, training_loss=0.003605481607376554, metrics={'train_runtime': 2253.3669, 'train_samples_per_second': 168.327, 'train_steps_per_second': 10.521, 'total_flos': 1.211275681328241e+16, 'train_loss': 0.003605481607376554, 'epoch': 2.0})

In [ ]:
trainer.evaluate(tokenized_test)

The repository for accuracy contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/accuracy.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


The repository for f1 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/f1.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


{'eval_loss': 0.0025527537800371647,
 'eval_accuracy': 0.999651997848714,
 'eval_f1': 0.6651762992341671,
 'eval_runtime': 495.2874,
 'eval_samples_per_second': 127.639,
 'eval_steps_per_second': 7.979,
 'epoch': 2.0}

In [ ]:
trainer.push_to_hub("anggari/distilnews")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

events.out.tfevents.1725836941.87594d3b309e.1250.0:   0%|          | 0.00/5.00k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

events.out.tfevents.1725837206.87594d3b309e.1250.1:   0%|          | 0.00/15.3k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/anggari/finetuning-sentiment-model-3000-samples/commit/48e725c5c4ac11b710e8a3db361fcc76318be972', commit_message='anggari/distilnews', commit_description='', oid='48e725c5c4ac11b710e8a3db361fcc76318be972', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub("distil_news_finetune")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/anggari/distil_news_finetune/commit/a96fd2b1e198146febe2758132cdf1a49280cad3', commit_message='Upload tokenizer', commit_description='', oid='a96fd2b1e198146febe2758132cdf1a49280cad3', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import pipeline

sentiment_model = pipeline(model="anggari/distil_news_finetune")

config.json:   0%|          | 0.00/769 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'LABEL_2', 'score': 0.9831281900405884},
 {'label': 'LABEL_2', 'score': 0.9881464242935181}]

In [ ]:
sentiment_model(["""
A busy bridge in northern Vietnam has collapsed in the wake of super typhoon Yagi, plunging 10 cars and two scooters into the Red River, Deputy Prime Minister Ho Duc Phoc said on Monday.
At least three people have so far been rescued and 13 are missing after the Phong Chau bridge in Phu Tho province collapsed, Mr Ho added. It is unclear if there have been any deaths.
Part of the 375-metre (1230 feet) structure is still standing, and the military has been instructed to build a pontoon bridge as soon as possible.
Yagi, Asia's most powerful storm this year, has killed more than 60 people since it made landfall in Vietnam on Saturday, bringing strong winds of up to 203 km/h (126 mph).
In the days since, the storm has wreaked havoc in northern Vietnam.At least 44 victims were killed by landslides and flash floods, the country's Ministry of Agriculture and Rural Development said on Monday - a 68-year-old woman, a one-year-old boy and a newborn baby among them.
More than 240 people have been injured, while some 1.5 million are still without power.
The typhoon also tore roofs from buildings and uprooted trees. Although it has weakened into a tropical depression, authorities have warned of more flooding and landslides as the storm moves westwards.
In the Yen Bai province, flood waters reached a metre (three feet) high on Monday, with 2,400 families moved to higher ground as the water levels rose, AFP news agency reported.
Yagi also sunk and swept adrift dozens of fishing boats. On Sunday, search and rescue personnel found 27 people drifting at sea after a dozen fishermen were reported missing.
Nearly 50,000 people have been evacuated from coastal towns in Vietnam, with authorities issuing a warning to remain indoors.
Schools were temporarily closed in 12 northern provinces, including Hanoi.
Before hitting Vietnam, Yagi left 24 people dead across southern China and the Philippines.
Scientists say typhoons and hurricanes are becoming stronger, more frequent and staying over land for longer due to climate change. Warmer ocean waters mean storms pick up more energy, which leads to higher wind speeds.
A warmer atmosphere also holds more moisture, which can lead to more intense rainfall.
"""
])

[{'label': 'LABEL_2', 'score': 0.9999912977218628}]